In [2]:
import tensorflow as tf

# CNN Layers

## Input Layer
This represents the input of a given image
* x: X size of image
* y: y size of image
* Depth: Number of params in each pixel
    * RGB: 3
    * Graysacle: 1
    
## Convolution layer(s)
Convultion layer = Number of filters * (Input Channels * Height * Width + 1)

* Number of filters: Amount of filters in a given layer
* Kernel Size: size of kernel in n x n
* Padding: What to do when we reach the end of the image
    * Valid: Stop
    * same: Add 0's to border of image
    
## Pooling layer(s)
These are typically added inbetween convolutional layers, more in depth info on pooling layers to follow
    
## Flatten
This takes our 3d tensor and turns it into an array

## Final layer(s)
Here we can treat the network like a traditional ANN

In [14]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(256,256,1)))

#In this case we have 8 filters, a 3x3 filter, and the relu actvation function
model.add(tf.keras.layers.Conv2D(8,3,strides=1, padding="valid", activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(5,5), strides=(5,5)))

model.add(tf.keras.layers.Conv2D(3,2,strides=1, padding="same", activation="relu"))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(2, activation="softmax"))

model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 254, 254, 8)    │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 50, 50, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 50, 50, 3)      │            99 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 7500)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │        15,002 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,181 (59.30 KB)

 Trainable params: 15,181 (59.30 KB)

 Non-trainable params: 0 (0.00 B)

# Pooling

Pooling layers pool local info to reduce dimensionality of intermediate convolutional outputs
This works similarly to a convolutional layer, except we return the max value of a patch in a filter instead of multiplying each patch

## Training
* ImageDataGenerator: Used for loading image data
    * rescale: normalizes our data 
    * zoom_range: used for zooming
    * rotation_range
    * width_shift_range
    * height_shift_range
* flow_from_directory: Loads each image from given directory
    * directory
    * class_mode: either categorical (automatically one hot encodes) or quantitative
    * color_mode: 
    * batch_size
    
 * model.compile: Creates our model from each layer
     * Optimizer: There are many to choose from, adam is very common
     * loss: Many to choose from, CategoricalCrossentropy is good for categorical data
     * metrics: this can be changed to view different metrics, I would leave this
 * model.fit: Runs our model
     * training_iterator: pass in the training iterator
     * steps_per_epoch: samples/batch_size
     * epochs: some integer
     * validation_steps: pass in the validation iterator
     * validation_steps: samples/batch_size

In [16]:
# This will not work as no local data is available
BATCH_SIZE = 5
training_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                                         zoom_range = 0.2,
                                                                         rotation_range = 15,
                                                                         width_shift_range = 0.05,
                                                                         height_shift_range=0.05)
training_iterator = training_data_generator.flow_from_directory('data/train',
                                                               class_mode = 'categorical',
                                                               color_mode = 'grayscale',
                                                               batch_size = 16)
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.005),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.AUC()]
)

model.fit(
    training_iterator,
    steps_per_epoch=training_iterator.samples/BATCH_SIZE,
    epochs=5,
    validation_data=validation_iterator,
    validation_steps=validation_iterator.samples/BATCH_SIZE)

FileNotFoundError: [Errno 2] No such file or directory: 'data/train'